In [1]:
source("http://bioconductor.org/biocLite.R")
biocLite("rhdf5")

Installing package into ‘/home/ilya/R/x86_64-pc-linux-gnu-library/3.3’
(as ‘lib’ is unspecified)
Bioconductor version 3.3 (BiocInstaller 1.22.3), ?biocLite for help
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.3 (BiocInstaller 1.22.3), R 3.3.1 (2016-06-21).
Installing package(s) ‘rhdf5’
also installing the dependency ‘zlibbioc’

Old packages: 'boot', 'MASS', 'spatial'


In [1]:
devtools::install_github("pachterlab/sleuth")

from URL https://api.github.com/repos/pachterlab/sleuth/zipball/master
Installing sleuth
Installing shiny
Installing httpuv
'/usr/lib/R/bin/R' --no-site-file --no-environ --no-save --no-restore --quiet  \
  CMD INSTALL '/tmp/RtmpMWwW2H/devtools4d42622c25bc/httpuv'  \
  --library='/home/ilya/R/x86_64-pc-linux-gnu-library/3.3' --install-tests 

Installing R6
'/usr/lib/R/bin/R' --no-site-file --no-environ --no-save --no-restore --quiet  \
  CMD INSTALL '/tmp/RtmpMWwW2H/devtools4d4245d9d26e/R6'  \
  --library='/home/ilya/R/x86_64-pc-linux-gnu-library/3.3' --install-tests 

'/usr/lib/R/bin/R' --no-site-file --no-environ --no-save --no-restore --quiet  \
  CMD INSTALL '/tmp/RtmpMWwW2H/devtools4d428a68d88/shiny'  \
  --library='/home/ilya/R/x86_64-pc-linux-gnu-library/3.3' --install-tests 

'/usr/lib/R/bin/R' --no-site-file --no-environ --no-save --no-restore --quiet  \
  CMD INSTALL  \
  '/tmp/RtmpMWwW2H/devtools4d421e6f6ad4/pachterlab-sleuth-048f055'  \
  --library='/home/ilya/R/x86_64-pc-l

In [2]:
library("sleuth")

Loading required package: ggplot2
Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [3]:
base_dir <- "../data/kallisto"
sample_id <- dir(file.path(base_dir), pattern='^lo')
sample_id

[1] "lo03" "lo05" "lo09" "lo11"

In [4]:
kal_dirs <- sapply(sample_id, function(id) file.path(base_dir, id))
kal_dirs

lo03                    lo05                    lo09 
"../data/kallisto/lo03" "../data/kallisto/lo05" "../data/kallisto/lo09" 
                   lo11 
"../data/kallisto/lo11"

In [5]:
dir(kal_dirs)

[1] "abundance.h5"  "abundance.h5"  "abundance.h5"  "abundance.h5" 
 [5] "abundance.tsv" "abundance.tsv" "abundance.tsv" "abundance.tsv"
 [9] "run_info.json" "run_info.json" "run_info.json" "run_info.json"

In [7]:
gsy_samples <- c('lo05', 'lo09', 'lo03', 'lo11')
gsy_samples

[1] "lo05" "lo09" "lo03" "lo11"

In [16]:
gsy_dirs <- c(kal_dirs[seq(1, length(kal_dirs), 2)], kal_dirs[seq(2, length(kal_dirs), 2)])
gsy_dirs


lo03                    lo09                    lo05 
"../data/kallisto/lo03" "../data/kallisto/lo09" "../data/kallisto/lo05" 
                   lo11 
"../data/kallisto/lo11"

In [ ]:
gsy

In [17]:
s2c <- data.frame(sample=c('lo03', 'lo05', 'lo09', 'lo11'), condition=c('gsy', 'wt', 'wt', 'gsy'), path=c('../data/kallisto/lo03','../data/kallisto/lo05', '../data/kallisto/lo09','../data/kallisto/lo11'))
s2c

sample,condition,path
lo03,gsy,../data/kallisto/lo03
lo05,wt,../data/kallisto/lo05
lo09,wt,../data/kallisto/lo09
lo11,gsy,../data/kallisto/lo11


In [18]:
so <- sleuth_prep(s2c, ~ condition)

reading in kallisto results
.

ERROR: Error: is(path, "character") is not TRUE


In [36]:
so <- sleuth_fit(so)

fitting measurement error models
shrinkage estimation
Adding missing grouping variables: `x_group`
computing variance of betas


In [37]:
so <- sleuth_fit(so, ~1, 'reduced')

fitting measurement error models
shrinkage estimation
Adding missing grouping variables: `x_group`
computing variance of betas


In [38]:
#so <- sleuth_test(so, which_beta='conditionwt')
so <- sleuth_lrt(so, 'reduced', 'full')

In [39]:
models(so)

[  full  ]
formula:  ~condition 
coefficients:
	(Intercept)
 	conditionwt
[  reduced  ]
formula:  ~1 
coefficients:
	(Intercept)


In [41]:
source("http://bioconductor.org/biocLite.R")
biocLite("biomaRt")

Bioconductor version 3.3 (BiocInstaller 1.22.3), ?biocLite for help
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.3 (BiocInstaller 1.22.3), R 3.3.1 (2016-06-21).
Installing package(s) ‘biomaRt’
also installing the dependencies ‘BiocGenerics’, ‘Biobase’, ‘IRanges’, ‘RSQLite’, ‘S4Vectors’, ‘XML’, ‘AnnotationDbi’

Old packages: 'boot', 'MASS', 'spatial'


In [42]:
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
  dataset = "celegans_gene_ensembl",
  host = 'ensembl.org')

In [43]:
t2g <- biomaRt::getBM(attributes = c("ensembl_transcript_id", "ensembl_gene_id",
    "external_gene_name"), mart = mart)
t2g <- dplyr::rename(t2g, target_id = ensembl_transcript_id,
  ens_gene = ensembl_gene_id, ext_gene = external_gene_name)
so <- sleuth_prep(s2c, ~ condition, target_mapping = t2g)

reading in kallisto results
......
normalizing est_counts
19296 targets passed the filter
normalizing tpm
merging in metadata
normalizing bootstrap samples
summarizing bootstraps


In [44]:
so <- sleuth_fit(so)

fitting measurement error models
shrinkage estimation
Adding missing grouping variables: `x_group`
computing variance of betas


In [46]:
so <- sleuth_fit(so, ~1, 'reduced')

fitting measurement error models
shrinkage estimation
Adding missing grouping variables: `x_group`
computing variance of betas


In [47]:
so <- sleuth_lrt(so, 'reduced', 'full')

In [51]:
so <- sleuth_wt(so, which_beta = 'conditionwt')

In [52]:
sleuth_live_settings(test_type='wt')

$test_type
[1] "wt"

In [ ]:
sleuth_live(so)


Listening on http://127.0.0.1:42427
Warning message:
: Removed 8 rows containing missing values (geom_point).Warning message:
: Removed 8 rows containing missing values (geom_point).

In [10]:
write.table(res,file='../kallisto/results.tsv', sep='\t')

In [ ]:
sleuth_live(so)

Loading required package: shiny

Listening on http://127.0.0.1:7861


In [ ]:
res <- sleuth_results(so, 'conditionwt')